<a href="https://colab.research.google.com/github/kplr-training/Airflow/blob/main/Ateliers/Solutions/06-Dynamic%20Task%20Mapping%20-%20Simple.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic Task Mapping : Création d’un DAG avec Simple task mapping

![simple](https://user-images.githubusercontent.com/123757632/231465297-6f71c9f6-468d-4afd-af7a-ed4d699ebd64.png)

1 . Définir le DAG (Directed Acyclic Graph) Airflow qui sera planifiée pour s'exécuter quotidiennement à partir d'une date  : 

Création du DAG nommée "single_map-download-ctx" qui est planifiée pour s'exécuter quotidiennement à partir du 8 avril 2023.

In [ ]:
from airflow import DAG
from airflow.decorators import task
from datetime import datetime
import pprint

with DAG('single_map-download-ctx', start_date=datetime(2023, 4, 8), schedule_interval='@daily', catchup=False) as dag:

2 . Le DAG contient deux tâches définies en utilisant le décorateur "@task" qui permet de les traiter comme des tâches indépendantes qui peuvent être exécutées en parallèle :

In [ ]:
@task
    def download_url(url: str):
        import requests
        response = requests.get(url)
        content = response.content.decode('utf-8')
        lines = content.splitlines()
        return(lines)

La première tâche est "download_url" qui prend une URL en entrée, utilise la bibliothèque "requests" pour télécharger le contenu du fichier CSV correspondant, le décode en utilisant l'encodage UTF-8 et le divise en une liste de lignes. La fonction retourne ensuite cette liste pour la passer en entrée de la tâche suivante.

In [ ]:
@task
    def print_content(content):                
        for file in content:
            print('---------------FILE-------------------')
            print(file)
            print('---------------------------------------')

La deuxième tâche est "print_content" qui prend le contenu renvoyé par la tâche précédente en entrée et affiche chaque fichier sur la console en ajoutant une ligne de séparation pour une meilleure lisibilité.

In [ ]:
files = download_url.expand(url=[
                                        "https://people.sc.fsu.edu/~jburkardt/data/csv/addresses.csv", 
                                        "https://people.sc.fsu.edu/~jburkardt/data/csv/grades.csv", 
                                        "https://people.sc.fsu.edu/~jburkardt/data/csv/taxables.csv"
                                        ])
    print_content(files)

Ensuite, l'utilisation de la méthode "expand" pour exécuter la tâche "download_url" trois fois en parallèle, une fois pour chaque URL spécifiée dans la liste "url" passée en paramètre. Les résultats de ces trois exécutions sont ensuite combinés en une seule liste qui est passée en entrée à la tâche "print_content".

![image](https://user-images.githubusercontent.com/123757632/231187559-53ca9c85-8c3b-4202-a67a-a4a1e16ef637.png)

Execution du Dag :

![image](https://user-images.githubusercontent.com/123757632/231190861-ee3a4544-d9ab-4db0-9791-555cc627c35b.png)

Verification du log : 

![image](https://user-images.githubusercontent.com/123757632/231454475-bd4dcab0-6374-4bf6-b01e-27458b16c932.png)


![image](https://user-images.githubusercontent.com/123757632/231454673-2dc96865-359e-41b5-b7e8-5e8f0c20f0b5.png)